In [1]:
#Importing liberaries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
!{sys.executable} -m pip install openpyxl

In [2]:
#Importing our dataset:
# Load all sheets at once into a dictionary
data = pd.read_excel("Superstore Sales fixed -DIVIDED tables.xlsx", sheet_name=None)

In [3]:
# Extract each sheet into its own DataFrame
orders = data['Orders']
customers = data['Customers']
products = data['Products']
sales = data['Sales']

In [4]:
#Checking for duplicates:
print(orders.duplicated().sum())
print(customers.duplicated().sum())
print(products.duplicated().sum())
print(sales.duplicated().sum())

0
0
0
1


In [6]:
#Dropping the duplicate found:
sales = sales.drop_duplicates()

In [7]:
#Confirming Duplicate is dropped:
print(sales.duplicated().sum())

0


In [8]:
# Checking for Nulls:
print("Orders nulls:\n", orders.isnull().sum(), "\n")
print("Customers nulls:\n", customers.isnull().sum(), "\n")
print("Products nulls:\n", products.isnull().sum(), "\n")
print("Sales nulls:\n", sales.isnull().sum(), "\n")

Orders nulls:
 Order ID       0
Order Date     0
Ship Date      0
Ship Mode      0
Customer ID    0
dtype: int64 

Customers nulls:
 Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Region           0
dtype: int64 

Products nulls:
 Category        0
Sub-Category    0
Product Name    0
Product ID      0
dtype: int64 

Sales nulls:
 Order ID      0
Sales         0
Product ID    0
dtype: int64 



In [9]:
# Convert 'Order Date' and 'Ship Date' to datetime
orders['Order Date'] = pd.to_datetime(orders['Order Date'], errors='coerce', dayfirst=False)
orders['Ship Date'] = pd.to_datetime(orders['Ship Date'], errors='coerce', dayfirst=False)

In [10]:
#unifying the dates format:
orders['Order Date'] = orders['Order Date'].dt.strftime('%d/%m/%Y')
orders['Ship Date'] = orders['Ship Date'].dt.strftime('%d/%m/%Y')

In [11]:
#Checking unification is done:
print(orders[['Order Date', 'Ship Date']].head())

   Order Date   Ship Date
0  11/08/2017  11/11/2017
1  12/05/2017  12/09/2017
2  06/12/2017  16/06/2017
3  08/01/2015  08/05/2015
4  10/11/2016  18/10/2016


In [13]:
# Merge Orders with Customers (each order gets its customer info)
orders_customers = orders.merge(customers, on='Customer ID', how='left')

# Merge Sales with Orders+Customers (each sale gets its order and customer info)
sales_orders_customers = sales.merge(orders_customers, on='Order ID', how='left')

# Merge with Products (each sale gets its product info)
full_sales_data = sales_orders_customers.merge(products, on='Product ID', how='left')

# Optional: check first few rows
print(full_sales_data.head())

# Optional: check columns to confirm all data is included
print(full_sales_data.columns)

         Order ID     Sales       Product ID  Order Date   Ship Date  \
0  CA-2017-152156   261.960  FUR-BO-10001798  11/08/2017  11/11/2017   
1  CA-2017-152156   731.940  FUR-CH-10000454  11/08/2017  11/11/2017   
2  CA-2017-114489  1951.840  FUR-CH-10000454  12/05/2017  12/09/2017   
3  CA-2017-138688    14.620  OFF-LA-10000240  06/12/2017  16/06/2017   
4  CA-2015-117345    17.544  OFF-LA-10000240  08/01/2015  08/05/2015   

        Ship Mode Customer ID    Customer Name    Segment        Country  \
0    Second Class    CG-12520      Claire Gute   Consumer  United States   
1    Second Class    CG-12520      Claire Gute   Consumer  United States   
2  Standard Class    JE-16165   Justin Ellison  Corporate  United States   
3    Second Class    DV-13045  Darrin Van Huff  Corporate  United States   
4  Standard Class    BF-10975   Barbara Fisher  Corporate  United States   

          City           State   Region         Category Sub-Category  \
0    Henderson        Kentucky    Sou

In [16]:
#Q1) Who are the top 10 customers in terms of total revenue?

top_customers = full_sales_data.groupby('Customer ID')['Sales'].sum()
top_customers = top_customers.to_frame().merge(customers[['Customer ID', 'Customer Name']], 
                                                on='Customer ID', how='left')
top_customers = top_customers.sort_values(by='Sales', ascending=False).head(10)
print(top_customers)

    Customer ID      Sales       Customer Name
700    SM-20320  25043.050         Sean Miller
741    TC-20980  19052.218        Tamara Chand
621    RB-19360  15117.339        Raymond Buch
730    TA-21385  14595.620        Tom Ashbrook
6      AB-10105  14473.571       Adrian Barton
434    KL-16645  14175.229        Ken Lonsdale
669    SC-20095  14142.334        Sanjit Chand
327    HL-15040  12873.298        Hunter Lopez
683    SE-20110  12209.438        Sanjit Engle
131    CC-12370  12129.072  Christopher Conant


In [17]:
#Q2) Who are the top 10 customers in terms of total orders?
top_customers_orders = orders.groupby('Customer ID')['Order ID'].nunique()
top_customers_orders = top_customers_orders.to_frame().rename(columns={'Order ID': 'Total Orders'})
top_customers_orders = top_customers_orders.sort_values(by='Total Orders', ascending=False).head(10)
print(top_customers_orders)

             Total Orders
Customer ID              
EP-13915               17
ZC-21910               13
CK-12205               13
JE-15745               13
PG-18820               13
SH-19975               12
RP-19390               12
SJ-20125               12
BP-11095               12
NS-18640               12


In [19]:
#Q3)How do customer segments differ in purchase frequency and order size?
# Merge Orders with Customers to get Segment info
orders_customers = pd.merge(orders, customers, on='Customer ID', how='left')

# Merge with Sales to get revenue info
orders_sales = pd.merge(orders_customers, sales, on='Order ID', how='left')

# Calculate total revenue per order
order_revenue = orders_sales.groupby('Order ID')['Sales'].sum().reset_index()

# Merge order revenue back to orders with segments
order_segment = pd.merge(order_revenue, orders_customers[['Order ID', 'Customer ID', 'Segment']], on='Order ID', how='left')

# Calculate metrics by Segment
segment_analysis = order_segment.groupby('Segment').agg({
    'Order ID': 'nunique',   # purchase frequency (number of orders)
    'Sales': 'mean'          # average order size
}).rename(columns={
    'Order ID': 'Total Orders',
    'Sales': 'Avg Order Size'
})

print(segment_analysis)

             Total Orders  Avg Order Size
Segment                                  
Consumer             2537      452.526816
Corporate            1491      461.766650
Home Office           894      475.056829


In [20]:
#Q4)Which customer segment (Consumer, Corporate, Home Office) generates the highest revenue?
# Merge Orders with Customers to get Segment info
orders_customers = pd.merge(orders, customers, on='Customer ID', how='left')

# Merge with Sales to get revenue info
orders_sales = pd.merge(orders_customers, sales, on='Order ID', how='left')

# Calculate total revenue per segment
segment_revenue = orders_sales.groupby('Segment')['Sales'].sum().sort_values(ascending=False)

print(segment_revenue)

Segment
Consumer       1.148061e+06
Corporate      6.884941e+05
Home Office    4.247008e+05
Name: Sales, dtype: float64


In [21]:
#Q5)What’s the average order value for each customer segment?
# Merge Orders with Customers to get Segment info
orders_customers = pd.merge(orders, customers, on='Customer ID', how='left')

# Merge with Sales to get revenue info
orders_sales = pd.merge(orders_customers, sales, on='Order ID', how='left')

# Calculate total sales and total orders per segment
segment_sales = orders_sales.groupby('Segment')['Sales'].sum()
segment_orders = orders_sales.groupby('Segment')['Order ID'].nunique()

# Calculate Average Order Value (AOV)
average_order_value = (segment_sales / segment_orders).sort_values(ascending=False)

print(average_order_value)

Segment
Home Office    475.056829
Corporate      461.766650
Consumer       452.526816
dtype: float64


In [33]:
#Q6)Which regions or cities have the most profitable customers?
# Merge the tables correctly using actual column names
merged_df = (
    sales
    .merge(orders, on="Order ID", how="left")
    .merge(customers, on="Customer ID", how="left")
)

# Group by Region, City, and Customer to calculate total sales
sales_by_location = (
    merged_df.groupby(["Region", "City", "Customer ID"], as_index=False)
    .agg({"Sales": "sum"})
)

# Sort to find the top customers by sales in each region
top_sales_customers = (
    sales_by_location
    .sort_values(["Region", "Sales"], ascending=[True, False])
)

# Show top 10 customers with highest sales overall
top_10_customers = (
    sales_by_location
    .sort_values("Sales", ascending=False)
    .head(10)
)
print("Top 10 Customers by Total Sales:")
print(top_10_customers)

# --- Visualization: Top 10 Cities by Total Sales ---
city_sales = (
    merged_df.groupby("City")["Sales"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

Top 10 Customers by Total Sales:
      Region           City Customer ID      Sales
501    South         Monroe    SM-20320  25043.050
774     West        Seattle    TC-20980  19052.218
177     East         Auburn    RB-19360  15117.339
314     East  New York City    TA-21385  14595.620
672     West        Phoenix    AB-10105  14473.571
29   Central        Chicago    KL-16645  14175.229
636     West    Los Angeles    SC-20095  14142.334
484    South      Jonesboro    HL-15040  12873.298
306     East  New York City    SE-20110  12209.438
448    South   Fayetteville    CC-12370  12129.072


In [34]:
#Q7)What’s the average order frequency per customer per year?
# Convert Order Date to datetime
orders["Order Date"] = pd.to_datetime(orders["Order Date"], errors="coerce")

# Extract the year
orders["Year"] = orders["Order Date"].dt.year

# Count orders per customer per year
orders_per_customer_year = (
    orders.groupby(["Customer ID", "Year"])["Order ID"]
    .nunique()  # unique orders per customer per year
    .reset_index(name="Order_Count")
)

# Calculate the average order frequency per customer per year
avg_order_freq = orders_per_customer_year["Order_Count"].mean()

print(f"Average order frequency per customer per year: {avg_order_freq:.2f}")

Average order frequency per customer per year: 1.36


In [35]:
#Q8)Which regions have the highest average sales per customer?
# Step 1: Join logic via relationships (Sales → Orders → Customers)
# We assume your environment already recognizes the relationships.

# Total sales per customer
customer_sales = (
    sales
    .join(orders.set_index("Order ID"), on="Order ID")       # Access related orders
    .join(customers.set_index("Customer ID"), on="Customer ID")  # Access related customers
    .groupby(["Region", "Customer ID"], as_index=False)
    .agg({"Sales": "sum"})
)

# Step 2: Average sales per customer in each region
avg_sales_per_customer_region = (
    customer_sales.groupby("Region", as_index=False)
    .agg({"Sales": "mean"})
    .rename(columns={"Sales": "Average Sales per Customer"})
    .sort_values("Average Sales per Customer", ascending=False)
)

print("Average Sales per Customer by Region:")
print(avg_sales_per_customer_region)

Average Sales per Customer by Region:
    Region  Average Sales per Customer
2    South                 2929.136196
3     West                 2862.986515
1     East                 2862.291556
0  Central                 2762.056641


In [37]:
#9)What is the repeated purchase per segment or region?
# Join through existing relationships (Orders → Customers)
orders_with_customers = (
    orders
    .join(customers.set_index("Customer ID"), on="Customer ID")
)

# Count how many orders each customer has made
customer_order_counts = (
    orders_with_customers.groupby(["Customer ID", "Segment", "Region"], as_index=False)
    .agg({"Order ID": "nunique"})
    .rename(columns={"Order ID": "Order_Count"})
)

# Identify repeat purchasers (Order_Count > 1)
customer_order_counts["Repeat_Purchase"] = customer_order_counts["Order_Count"] > 1

# ---- Repeated Purchase per Segment ----
repeat_rate_segment = (
    customer_order_counts.groupby("Segment")["Repeat_Purchase"]
    .mean()
    .reset_index(name="Repeat Purchase Rate")
    .sort_values("Repeat Purchase Rate", ascending=False)
)

# ---- Repeated Purchase per Region ----
repeat_rate_region = (
    customer_order_counts.groupby("Region")["Repeat_Purchase"]
    .mean()
    .reset_index(name="Repeat Purchase Rate")
    .sort_values("Repeat Purchase Rate", ascending=False)
)

print("Repeated Purchase Rate per Segment:")
print(repeat_rate_segment)
print("\nRepeated Purchase Rate per Region:")
print(repeat_rate_region)

Repeated Purchase Rate per Segment:
       Segment  Repeat Purchase Rate
2  Home Office              0.993243
0     Consumer              0.982885
1    Corporate              0.978814

Repeated Purchase Rate per Region:
    Region  Repeat Purchase Rate
3     West              0.992095
0  Central              0.988571
1     East              0.978541
2    South              0.969697


In [38]:
#Q10)Which cities have the highest customer retention or loyalty trends (by order frequency)?
# Link Orders with Customers using the relationships
orders_with_customers = (
    orders
    .join(customers.set_index("Customer ID"), on="Customer ID")
)

# Count how many orders each customer has made
customer_order_counts = (
    orders_with_customers.groupby(["Customer ID", "City"], as_index=False)
    .agg({"Order ID": "nunique"})
    .rename(columns={"Order ID": "Order_Count"})
)

# --- City-level metrics ---

# 1️⃣ Average order frequency per customer in each city
avg_order_freq_city = (
    customer_order_counts.groupby("City", as_index=False)
    .agg({"Order_Count": "mean"})
    .rename(columns={"Order_Count": "Average Orders per Customer"})
    .sort_values("Average Orders per Customer", ascending=False)
)

# 2️⃣ (Optional) Retention rate: % of repeat customers in each city
customer_order_counts["Repeat_Customer"] = customer_order_counts["Order_Count"] > 1

retention_rate_city = (
    customer_order_counts.groupby("City", as_index=False)["Repeat_Customer"]
    .mean()
    .rename(columns={"Repeat_Customer": "Retention Rate"})
    .sort_values("Retention Rate", ascending=False)
)

print("Average Orders per Customer by City:")
print(avg_order_freq_city.head(10))

print("\nCustomer Retention Rate by City:")
print(retention_rate_city.head(10))

Average Orders per Customer by City:
              City  Average Orders per Customer
138          Miami                         13.0
70         Fremont                         12.0
15    Bossier City                         12.0
229      Urbandale                         12.0
152  New Brunswick                         12.0
126        Madison                         12.0
175    Pico Rivera                         11.0
80      Greenville                         11.0
81      Grove City                         11.0
48   Daytona Beach                         10.0

Customer Retention Rate by City:
            City  Retention Rate
0          Akron             1.0
1    Albuquerque             1.0
2       Amarillo             1.0
3   Apple Valley             1.0
4      Arlington             1.0
6         Arvada             1.0
7      Asheville             1.0
9         Auburn             1.0
8        Atlanta             1.0
10        Aurora             1.0


In [39]:
#Q11)What is the total number of customers filtered by each regions?
# Count the total number of unique customers in each region
customers_per_region = (
    customers.groupby("Region")["Customer ID"]
    .nunique()
    .reset_index(name="Total Customers")
    .sort_values("Total Customers", ascending=False)
)

print("Total Number of Customers by Region:")
print(customers_per_region)

Total Number of Customers by Region:
    Region  Total Customers
3     West              253
1     East              233
0  Central              175
2    South              132


In [40]:
#Q12)How many customers in each state?
# Count unique customers in each state
customers_per_state = (
    customers.groupby("State")["Customer ID"]
    .nunique()
    .reset_index(name="Total Customers")
    .sort_values("Total Customers", ascending=False)
)

print("Total Number of Customers by State:")
print(customers_per_state)

Total Number of Customers by State:
                   State  Total Customers
3             California              157
27              New York               95
36                 Texas               72
32          Pennsylvania               55
10              Illinois               40
29                  Ohio               38
39            Washington               37
28        North Carolina               32
4               Colorado               23
8                Florida               22
1                Arizona               20
38              Virginia               18
9                Georgia               17
18              Michigan               15
19             Minnesota               13
14              Kentucky               12
35             Tennessee               11
25            New Jersey                9
11               Indiana                9
40             Wisconsin                9
17         Massachusetts                8
0                Alabama                

In [42]:
#Q3)What customer segments are most profitable (e.g., by region,)?
# Use the relationships to access related data
sales_with_details = (
    sales
    .join(orders.set_index("Order ID"), on="Order ID")
    .join(customers.set_index("Customer ID"), on="Customer ID")
)

# Step 1: Total sales by Segment and Region
segment_region_sales = (
    sales_with_details.groupby(["Segment", "Region"], as_index=False)
    .agg({"Sales": "sum"})
    .rename(columns={"Sales": "Total Sales"})
    .sort_values("Total Sales", ascending=False)
)

print("Total Sales by Segment and Region:")
print(segment_region_sales)

Total Sales by Segment and Region:
        Segment   Region  Total Sales
3      Consumer     West  408547.5345
1      Consumer     East  336723.1327
0      Consumer  Central  232116.0912
5     Corporate     East  205587.8464
7     Corporate     West  197214.9349
2      Consumer    South  170673.7726
4     Corporate  Central  163542.6432
9   Home Office     East  124602.9534
6     Corporate    South  122148.6503
11  Home Office     West  118573.1188
10  Home Office    South   93823.5550
8   Home Office  Central   87701.1777


In [43]:
##Product analysis##
#Q1) What are the top 10 best-selling products?
# Merge Sales with Products to get product details
sales_products = pd.merge(sales, products, on='Product ID', how='left')

# Group by Product Name and sum the sales
top_products = sales_products.groupby('Product Name')['Sales'].sum().sort_values(ascending=False).head(10)

print(top_products)

Product Name
Canon imageCLASS 2200 Advanced Copier                                          61599.824
Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind    27453.384
Cisco TelePresence System EX90 Videoconferencing Unit                          22638.480
HON 5400 Series Task Chairs for Big and Tall                                   21870.576
GBC DocuBind TL300 Electric Binding System                                     19823.479
GBC Ibimaster 500 Manual ProClick Binding System                               19024.500
Hewlett Packard LaserJet 3310 Copier                                           18839.686
HP Designjet T520 Inkjet Large Format Printer - 24" Color                      18374.895
GBC DocuBind P400 Electric Binding System                                      17965.068
High Speed Automatic Electric Letter Opener                                    17030.312
Name: Sales, dtype: float64


In [53]:
# Q2: Which product categories contribute the most to total revenue?

category_revenue = (
    full_sales_data.groupby("Category", as_index=False)["Sales"]
    .sum()
    .sort_values("Sales", ascending=False)
)

print("Total Revenue by Product Category:")
print(category_revenue)

Total Revenue by Product Category:
          Category        Sales
2       Technology  827455.8730
0        Furniture  728377.2037
1  Office Supplies  705422.3340


In [25]:
#Q3) Which product categories and sub-categories generate the highest sales?
# Merge Sales with Products to get category and sub-category info
sales_products = pd.merge(sales, products, on='Product ID', how='left')

# Group by Category and Sub-Category, then sum the Sales
category_sales = (
    sales_products.groupby(['Category', 'Sub-Category'])['Sales']
    .sum()
    .sort_values(ascending=False)
)

print(category_sales)

Category         Sub-Category
Technology       Phones          327782.4480
Furniture        Chairs          322541.3590
Office Supplies  Storage         219343.3920
Furniture        Tables          202810.6280
Office Supplies  Binders         200028.7850
Technology       Machines        189238.6310
                 Accessories     164186.7000
                 Copiers         146248.0940
Furniture        Bookcases       113813.1987
Office Supplies  Appliances      104618.4030
Furniture        Furnishings      89212.0180
Office Supplies  Paper            76828.3040
                 Supplies         46420.3080
                 Art              26705.4100
                 Envelopes        16128.0460
                 Labels           12347.7260
                 Fasteners         3001.9600
Name: Sales, dtype: float64


In [52]:
# Q4: How does product performance vary across regions and customer segments?

product_performance = (
    full_sales_data.groupby(["Region", "Segment", "Category"], as_index=False)["Sales"]
    .sum()
    .sort_values(["Region", "Segment", "Sales"], ascending=[True, True, False])
)

print("Product performance (total Sales) by Region and Segment:")
print(product_performance)

Product performance (total Sales) by Region and Segment:
     Region      Segment         Category        Sales
1   Central     Consumer  Office Supplies   85242.3710
0   Central     Consumer        Furniture   80695.2882
2   Central     Consumer       Technology   66178.4320
5   Central    Corporate       Technology   62580.5730
3   Central    Corporate        Furniture   54132.1032
4   Central    Corporate  Office Supplies   46829.9670
8   Central  Home Office       Technology   32669.4370
7   Central  Home Office  Office Supplies   30526.7800
6   Central  Home Office        Furniture   24504.9607
11     East     Consumer       Technology  142794.1380
9      East     Consumer        Furniture  101925.0427
10     East     Consumer  Office Supplies   92003.9520
14     East    Corporate       Technology   69867.0960
12     East    Corporate        Furniture   68881.0954
13     East    Corporate  Office Supplies   66839.6550
17     East  Home Office       Technology   58625.7010
16     E

In [58]:
# Q5: Which product categories show the fastest sales growth over time?
# Ensure Order Date is datetime and extract the year
full_sales_data["Order Date"] = pd.to_datetime(full_sales_data["Order Date"], errors="coerce")
full_sales_data["Order Year"] = full_sales_data["Order Date"].dt.year

# Calculate total sales per Category per Year
category_year_sales = (
    full_sales_data.groupby(["Category", "Order Year"], as_index=False)["Sales"]
    .sum()
    .sort_values(["Category", "Order Year"])
)

# Calculate year-over-year growth percentage
category_year_sales["Sales Growth %"] = (
    category_year_sales.groupby("Category")["Sales"].pct_change() * 100
)

# Find categories with the fastest growth in the most recent year
latest_growth = (
    category_year_sales.sort_values(["Category", "Order Year"], ascending=[True, False])
    .drop_duplicates("Category")
    .sort_values("Sales Growth %", ascending=False)
)

# Display results
print("Categories with the Fastest Sales Growth (Most Recent Year):")
print(latest_growth[["Category", "Order Year", "Sales", "Sales Growth %"]])

Categories with the Fastest Sales Growth (Most Recent Year):
           Category  Order Year       Sales  Sales Growth %
7   Office Supplies      2018.0  85966.9020       23.000652
3         Furniture      2018.0  82418.3183        3.827861
11       Technology      2018.0  98167.8540      -16.083952


In [59]:
# Q6: Visualize the contribution of each Sub-Category to overall revenue

# Group by Sub-Category to calculate total revenue
subcat_revenue = (
    full_sales_data.groupby(["Category", "Sub-Category"], as_index=False)["Sales"]
    .sum()
    .sort_values("Sales", ascending=False)
)

print("Total Revenue by Sub-Category:")
print(subcat_revenue)

Total Revenue by Sub-Category:
           Category Sub-Category        Sales
16       Technology       Phones  327782.4480
1         Furniture       Chairs  322541.3590
11  Office Supplies      Storage  219343.3920
3         Furniture       Tables  202810.6280
6   Office Supplies      Binders  200028.7850
15       Technology     Machines  189238.6310
13       Technology  Accessories  164186.7000
14       Technology      Copiers  146248.0940
0         Furniture    Bookcases  113813.1987
4   Office Supplies   Appliances  104618.4030
2         Furniture  Furnishings   89212.0180
10  Office Supplies        Paper   76828.3040
12  Office Supplies     Supplies   46420.3080
5   Office Supplies          Art   26705.4100
7   Office Supplies    Envelopes   16128.0460
9   Office Supplies       Labels   12347.7260
8   Office Supplies    Fasteners    3001.9600


In [61]:
# Q7: Most frequently ordered products per Region or Customer Segment

# Count the number of orders per product in each Region and Segment
product_frequency = (
    full_sales_data.groupby(["Region", "Segment", "Product Name"], as_index=False)
    .agg({"Order ID": "nunique"})   # count distinct orders per product
    .rename(columns={"Order ID": "Order_Count"})
)

# Sort by most ordered products
product_frequency = product_frequency.sort_values(
    ["Region", "Segment", "Order_Count"], ascending=[True, True, False]
)

print("Most frequently ordered products per Region and Segment:")
print(product_frequency.head(10))  # display top 10 results

Most frequently ordered products per Region and Segment:
      Region   Segment                                       Product Name  \
654  Central  Consumer                            Storex Dura Pro Binders   
256  Central  Consumer                                  Easy-staple paper   
405  Central  Consumer                Ibico Hi-Tech Manual Binding System   
615  Central  Consumer  SanDisk Ultra 32 GB MicroSDHC Class 10 Memory ...   
646  Central  Consumer                                    Staple envelope   
650  Central  Consumer                                            Staples   
652  Central  Consumer               Sterilite Officeware Hinged File Box   
9    Central  Consumer  3.6 Cubic Foot Counter Height Office Refrigerator   
13   Central  Consumer                   ACCOHIDE 3-Ring Binder, Blue, 1"   
16   Central  Consumer                      AT&T 17929 Lendline Telephone   

     Order_Count  
654            6  
256            4  
405            4  
615            4  


In [62]:
##Orders & Sales analysis:##
# Q1: Total Sales and Average Sales per Order?

#Total sales
total_sales = full_sales_data["Sales"].sum()

#Average sales per order 
avg_sales_per_order = (
    full_sales_data.groupby("Order ID")["Sales"].sum().mean()
)

print(f"Total Sales: {total_sales:,.2f}")
print(f"Average Sales per Order: {avg_sales_per_order:,.2f}")

Total Sales: 2,261,255.41
Average Sales per Order: 459.42


In [63]:
# Q2: How many orders have been placed in every year?

#Convert Order Date to datetime (if not already)
full_sales_data["Order Date"] = pd.to_datetime(full_sales_data["Order Date"])

#Extract year from Order Date
full_sales_data["Order Year"] = full_sales_data["Order Date"].dt.year

#Count unique orders per year
orders_per_year = (
    full_sales_data.groupby("Order Year")["Order ID"]
    .nunique()
    .reset_index()
    .rename(columns={"Order ID": "Number of Orders"})
    .sort_values("Order Year")
)

print("Number of Orders per Year:")
print(orders_per_year)

Number of Orders per Year:
   Order Year  Number of Orders
0      2015.0               375
1      2016.0               408
2      2017.0               520
3      2018.0               672


In [64]:
# Q3: What is the distribution of orders that are mostly purchased by region?

#Count unique orders per Region
orders_by_region = (
    full_sales_data.groupby("Region")["Order ID"]
    .nunique()
    .reset_index()
    .rename(columns={"Order ID": "Number of Orders"})
    .sort_values("Number of Orders", ascending=False)
)

print("Number of Orders by Region:")
print(orders_by_region)

Number of Orders by Region:
    Region  Number of Orders
3     West              1528
1     East              1466
0  Central              1136
2    South               792


In [73]:
# Q4: Average Delivery Time (difference between Order Date & Ship Date)?

# --- Step 1: Convert columns properly to datetime ---
full_sales_data["Order Date"] = pd.to_datetime(full_sales_data["Order Date"], errors="coerce")
full_sales_data["Ship Date"] = pd.to_datetime(full_sales_data["Ship Date"], errors="coerce")

# --- Step 2: Drop rows where conversion failed (if any nulls) ---
full_sales_data = full_sales_data.dropna(subset=["Order Date", "Ship Date"])

# --- Step 3: Calculate delivery time in days ---
full_sales_data["Delivery_Days"] = (
    full_sales_data["Ship Date"] - full_sales_data["Order Date"]
).dt.days  

# --- Step 4: Calculate averages ---
avg_delivery_time = full_sales_data["Delivery_Days"].mean()

print(f"Average Delivery Time: {avg_delivery_time:.2f} days")

# --- Optional: Breakdown by Region ---
avg_delivery_region = (
    full_sales_data.groupby("Region")["Delivery_Days"].mean().reset_index()
)
print("\nAverage Delivery Time by Region:")
print(avg_delivery_region)

Average Delivery Time: 3.52 days

Average Delivery Time by Region:
    Region  Delivery_Days
0  Central       3.429688
1     East       3.438762
2    South       3.653933
3     West       3.577830


In [74]:
# Q5: What’s the average order frequency per customer per year?

#Ensure Order Date is in datetime format
full_sales_data["Order Date"] = pd.to_datetime(full_sales_data["Order Date"], errors="coerce")

#Extract Year from Order Date
full_sales_data["Order Year"] = full_sales_data["Order Date"].dt.year

#Count unique orders per customer per year
orders_per_customer_year = (
    full_sales_data.groupby(["Customer ID", "Order Year"])["Order ID"]
    .nunique()
    .reset_index()
    .rename(columns={"Order ID": "Orders_Per_Year"})
)

#Calculate average order frequency per customer per year
avg_order_freq = orders_per_customer_year["Orders_Per_Year"].mean()

print(f"Average Order Frequency per Customer per Year: {avg_order_freq:.2f} orders")

Average Order Frequency per Customer per Year: 1.22 orders


In [75]:
# Q6: What is the total sales revenue over time (quarterly and annually)?

#Ensure Order Date is in datetime format
full_sales_data["Order Date"] = pd.to_datetime(full_sales_data["Order Date"], errors="coerce")

#Extract Year and Quarter
full_sales_data["Order Year"] = full_sales_data["Order Date"].dt.year
full_sales_data["Order Quarter"] = full_sales_data["Order Date"].dt.to_period("Q")

#Total sales by Year
annual_sales = (
    full_sales_data.groupby("Order Year")["Sales"]
    .sum()
    .reset_index()
    .rename(columns={"Sales": "Total Sales"})
)

#Total sales by Quarter
quarterly_sales = (
    full_sales_data.groupby("Order Quarter")["Sales"]
    .sum()
    .reset_index()
    .rename(columns={"Sales": "Total Sales"})
)

# ---------- Display ----------
print("Total Annual Sales:")
print(annual_sales)

print("\nTotal Quarterly Sales:")
print(quarterly_sales.head(12))  

Total Annual Sales:
   Order Year  Total Sales
0        2015  109305.6880
1        2016  115481.6202
2        2017  198238.0994
3        2018  196142.8163

Total Quarterly Sales:
   Order Quarter  Total Sales
0         2015Q1   12489.6150
1         2015Q2   26306.7170
2         2015Q3   32013.4340
3         2015Q4   38495.9220
4         2016Q1   15373.7450
5         2016Q2   18172.5420
6         2016Q3   23699.4542
7         2016Q4   58235.8790
8         2017Q1   40826.0610
9         2017Q2   31722.3540
10        2017Q3   42002.6846
11        2017Q4   83686.9998


In [76]:
# Q7: Does Ship Mode affect total sales or delivery speed?

#Ensure Order Date and Ship Date are datetime
full_sales_data["Order Date"] = pd.to_datetime(full_sales_data["Order Date"], errors="coerce")
full_sales_data["Ship Date"] = pd.to_datetime(full_sales_data["Ship Date"], errors="coerce")

#Calculate delivery time in days (if not already done)
if "Delivery_Days" not in full_sales_data.columns:
    full_sales_data["Delivery_Days"] = (
        full_sales_data["Ship Date"] - full_sales_data["Order Date"]
    ).dt.days

#Group by Ship Mode to calculate total sales and average delivery time
shipmode_analysis = (
    full_sales_data.groupby("Ship Mode")
    .agg(
        Total_Sales=("Sales", "sum"),
        Average_Delivery_Time=("Delivery_Days", "mean"),
        Number_of_Orders=("Order ID", "nunique")
    )
    .reset_index()
    .sort_values("Total_Sales", ascending=False)
)

print("Impact of Ship Mode on Sales and Delivery Speed:")
print(shipmode_analysis)

Impact of Ship Mode on Sales and Delivery Speed:
        Ship Mode  Total_Sales  Average_Delivery_Time  Number_of_Orders
3  Standard Class  322023.3653               4.787879               691
2    Second Class  124312.3942               3.188571               255
0     First Class  115188.9864               2.068702               272
1        Same Day   57643.4780               0.066390               114


In [77]:
# Q9: How do sales fluctuate over time (monthly, quarterly, yearly)?

#Ensure Order Date is in datetime format
full_sales_data["Order Date"] = pd.to_datetime(full_sales_data["Order Date"], errors="coerce")

#Extract Year, Quarter, and Month for analysis
full_sales_data["Year"] = full_sales_data["Order Date"].dt.year
full_sales_data["Quarter"] = full_sales_data["Order Date"].dt.to_period("Q")
full_sales_data["Month"] = full_sales_data["Order Date"].dt.to_period("M")

# ----------  Total Sales per Year ----------
sales_yearly = full_sales_data.groupby("Year", as_index=False)["Sales"].sum()

# ---------- Total Sales per Quarter ----------
sales_quarterly = full_sales_data.groupby("Quarter", as_index=False)["Sales"].sum()

# ----------  Total Sales per Month ----------
sales_monthly = full_sales_data.groupby("Month", as_index=False)["Sales"].sum()

#Display tables
print("Total Sales by Year:")
print(sales_yearly)
print("\nTotal Sales by Quarter:")
print(sales_quarterly)
print("\nTotal Sales by Month:")
print(sales_monthly.head(12))  

Total Sales by Year:
   Year        Sales
0  2015  109305.6880
1  2016  115481.6202
2  2017  198238.0994
3  2018  196142.8163

Total Sales by Quarter:
   Quarter       Sales
0   2015Q1  12489.6150
1   2015Q2  26306.7170
2   2015Q3  32013.4340
3   2015Q4  38495.9220
4   2016Q1  15373.7450
5   2016Q2  18172.5420
6   2016Q3  23699.4542
7   2016Q4  58235.8790
8   2017Q1  40826.0610
9   2017Q2  31722.3540
10  2017Q3  42002.6846
11  2017Q4  83686.9998
12  2018Q1  21455.6710
13  2018Q2  38012.4403
14  2018Q3  46630.8920
15  2018Q4  90043.8130

Total Sales by Month:
      Month      Sales
0   2015-01   4818.302
1   2015-02   1437.434
2   2015-03   6233.879
3   2015-04  11793.470
4   2015-05   4331.647
5   2015-06  10181.600
6   2015-07   4414.836
7   2015-08   8695.831
8   2015-09  18902.767
9   2015-10   3441.627
10  2015-11  20806.321
11  2015-12  14247.974


In [83]:
# Q10: Are there any seasonal patterns (e.g., sales peaks around holidays)?

#Ensure Order Date is datetime
full_sales_data["Order Date"] = pd.to_datetime(full_sales_data["Order Date"], errors="coerce")

#Extract Month and Year
full_sales_data["Year"] = full_sales_data["Order Date"].dt.year
full_sales_data["Month"] = full_sales_data["Order Date"].dt.month
full_sales_data["Month_Name"] = full_sales_data["Order Date"].dt.strftime("%B")

# Group by month number to maintain correct order
monthly_avg_sales = (
    full_sales_data.groupby(["Month", "Month_Name"], as_index=False)["Sales"]
    .mean()
    .sort_values(by="Month")
)

print("Average Monthly Sales (Seasonal Trend):")
print(monthly_avg_sales)

# ---------- OPTIONAL: Yearly comparison ----------
monthly_sales_yearly = (
    full_sales_data.groupby(["Year", "Month", "Month_Name"], as_index=False)["Sales"]
    .sum()
    .sort_values(by=["Year", "Month"])
)

print("\nMonthly Total Sales by Year:")
print(monthly_sales_yearly)

Average Monthly Sales (Seasonal Trend):
    Month Month_Name       Sales
0       1    January  289.525713
1       2   February  255.939966
2       3      March  240.206457
3       4      April  223.349095
4       5        May  199.983517
5       6       June  209.021801
6       7       July  199.416893
7       8     August  189.357662
8       9  September  211.906978
9      10    October  279.257367
10     11   November  262.562235
11     12   December  230.326243

Monthly Total Sales by Year:
    Year  Month Month_Name       Sales
0   2015      1    January   4818.3020
1   2015      2   February   1437.4340
2   2015      3      March   6233.8790
3   2015      4      April  11793.4700
4   2015      5        May   4331.6470
5   2015      6       June  10181.6000
6   2015      7       July   4414.8360
7   2015      8     August   8695.8310
8   2015      9  September  18902.7670
9   2015     10    October   3441.6270
10  2015     11   November  20806.3210
11  2015     12   December  14247

In [26]:
#Most shipment mode used?
# Count how many times each ship mode appears
most_used_ship_mode = orders['Ship Mode'].value_counts()

print(most_used_ship_mode)
print(most_used_ship_mode.idxmax())

Ship Mode
Standard Class    2945
Second Class       944
First Class        772
Same Day           261
Name: count, dtype: int64
Standard Class


In [27]:
#Top 10 Sales states?
# Merge Orders with Customers to get state info
orders_customers = pd.merge(orders, customers, on='Customer ID', how='left')

# Merge with Sales to get revenue info
orders_sales = pd.merge(orders_customers, sales, on='Order ID', how='left')

# Group by State and sum the sales
top_states = (
    orders_sales.groupby('State')['Sales']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

print(top_states)

State
California        415078.9677
New York          319147.3945
Texas             162876.0813
Pennsylvania      156123.6060
Washington        130274.2928
North Carolina    121877.5001
Illinois          121281.5725
Arizona            71852.5210
Ohio               69322.1550
Colorado           63034.5462
Name: Sales, dtype: float64


In [85]:
# Q: How many orders are shipped above the average shipment time?

# ✅ Ensure date columns are datetime
full_sales_data["Order Date"] = pd.to_datetime(full_sales_data["Order Date"], errors="coerce")
full_sales_data["Ship Date"] = pd.to_datetime(full_sales_data["Ship Date"], errors="coerce")

# ✅ Calculate delivery time in days
full_sales_data["Delivery_Days"] = (
    full_sales_data["Ship Date"] - full_sales_data["Order Date"]
).dt.days

# ✅ Calculate the average delivery time
average_delivery_time = full_sales_data["Delivery_Days"].mean()

# ✅ Count how many orders took longer than the average
above_average_shipments = full_sales_data[full_sales_data["Delivery_Days"] > average_delivery_time].shape[0]

# ✅ Print results
print(f"Average delivery time: {average_delivery_time:.2f} days")
print(f"Number of orders shipped above average time: {above_average_shipments}")

Average delivery time: 3.52 days
Number of orders shipped above average time: 1581
